In [1]:
import pandas as pd
from parse import parse_htmls
from util import read_place_desc, read_htmls_in, write_places_to_tsv, read_places
from service import PlaceService, SearchEngine
from index import preprocess, Index, TfIdfIndex

import requests
from tqdm import tqdm
import os
import time
import util

# Data collection
##  1.1. Get the list of places





In [ ]:
with open('most_popular_places.txt', 'w') as file:
    for i in tqdm(range(300)):
        
        url = 'https://www.atlasobscura.com/places?page='+str(i+1)+'&sort=likes_count'
        list_page = requests.get(url)
        list_soup = BeautifulSoup(list_page.text)
        list_places = [x['href'] for x in list_soup.find_all('a', {'class':"content-card content-card-place"})]
        for place in tqdm(list_places):
            file.write('https://www.atlasobscura.com' + str(place))
            file.write('\n')

## 1.2. Crawl places

In [ ]:
f= open('most_popular_places.txt', 'r')
for j, url in enumerate(f):
    
    if j %18 == 0:
        dir_path = f'page{j//18+1}'
        if not os.path.isdir(dir_path):
            os.mkdir(dir_path)
            
    response = requests.get(url.strip())
    with open(f'{dir_path}/{j+1}.html','w') as file:
        print()
        file.write(response.text)
        
    time.sleep(1)

# 1.3 Parse Pages

In [3]:
all_htmls = read_htmls_in('./pages')

In [4]:
all_places = parse_htmls(all_htmls)

In [263]:
%store htmls

Stored 'htmls' (list)


In [4]:
%store -r htmls

In [2]:
#tsv_path = write_places_to_tsv('./', all_places)
names, descriptions = read_place_desc('./places.tsv')
place_service = PlaceService()
place_service.load('./places.tsv')

# Create Conjunctive Index

In [3]:
index = Index.create_from(names, descriptions)

# Create Search Engine

In [4]:
search_engine = SearchEngine(index, place_service)

# Run Search

In [5]:
search_engine.query('american museum')

,name,desc,url
1072,Early Television Museum,Steve McVoy was always fascinated by TV. In mi...,https://www.atlasobscura.com/places/early-tele...
4954,Madam C.J. Walker Museum &amp; WERD Radio,"This small brick building, located just a few ...",https://www.atlasobscura.com/places/madam-cj-w...
569,American Museum of Magic,"Presto Change-o! Founded in 1978, the American...",https://www.atlasobscura.com/places/american-m...
4571,Deutsches Jagd und Fischereimuseum (German Mus...,With its rows of taxidermy animal heads and di...,https://www.atlasobscura.com/places/deutsches-...
904,Lunch Box Museum,Lunch boxes bring back a certain sense of nost...,https://www.atlasobscura.com/places/lunch-box-...
...,...,...,...
3250,Bare-Chested George Washington,"At first glance, the Smithsonian’s bare-cheste...",https://www.atlasobscura.com/places/bare-chest...
7118,Castle Post,Looking more like an abandoned Medieval Times ...,https://www.atlasobscura.com/places/castle-post
5429,Old Time Wooden Nickel Company,"The adage goes, “don’t take any wooden nickels...",https://www.atlasobscura.com/places/old-time-w...
456,Mildred E. Mathias Botanical Garden,"Founded in 1929, the botanical garden at Unive...",https://www.atlasobscura.com/places/mildred-e-...


# Create TF-IDF Index

In [3]:
tf_idf_index = TfIdfIndex.create_from(names, descriptions)

# Create TF-IDF Search Engine

In [4]:
tf_idf_search_engine = SearchEngine(tf_idf_index, place_service)

In [5]:
tf_idf_search_engine.query_top_k("american museum", 10)

,name,desc,url,similarity
1670,V. C. Morris Gift Shop,"This building, on Maiden Lane in downtown San ...",https://www.atlasobscura.com/places/v-c-morris...,1.0
2802,The Hobo Museum,"“Decide your own life, don’t let another perso...",https://www.atlasobscura.com/places/the-hobo-m...,1.0
6885,The Carson Mansion,"Designed by Samuel and Joseph Newsom, architec...",https://www.atlasobscura.com/places/the-carson...,1.0
7043,Dumbarton Oaks,This stunning estate with sprawling formal gar...,https://www.atlasobscura.com/places/dumbarton-...,1.0
1317,Aria Antiques,Tucked between the Italian bakeries and laundr...,https://www.atlasobscura.com/places/aria-antiques,1.0
4382,Warther Museum,When Ernest “Mooney” Warther was a boy growing...,https://www.atlasobscura.com/places/warther-mu...,1.0
1804,Uncommon Objects,Like an elegant antiques mall gone horribly wr...,https://www.atlasobscura.com/places/uncommon-o...,1.0
2523,Treasures of the Steamboat Arabia,When the mighty Steamboat Arabia sank near Kan...,https://www.atlasobscura.com/places/treasures-...,1.0
5999,The Witch House of Salem,The Salem witchcraft trials took place between...,https://www.atlasobscura.com/places/witch-hous...,1.0
5778,Zwack Unicum Museum,Sometimes referred to as the Hungarian nationa...,https://www.atlasobscura.com/places/zwack-unic...,1.0


In [20]:
result_dense_list

[[0.6455461258874972,
  0.0036064029379189785,
  0.0036064029379189785,
  0.02163841762751387,
  0.0018032014689594892,
  0.027048022034392338,
  0.0036064029379189785,
  0.007212805875837957,
  0.0036064029379189785,
  0.0018032014689594892,
  0.0036064029379189785,
  0.0018032014689594892,
  0.0018032014689594892,
  0.0018032014689594892,
  0.009016007344797445,
  0.0018032014689594892,
  0.0036064029379189785,
  0.01803201468959489,
  0.014425611751675914,
  0.0018032014689594892,
  0.009016007344797445,
  0.010819208813756935,
  0.0036064029379189785,
  0.0018032014689594892,
  0.005409604406878468,
  0.0018032014689594892,
  0.005409604406878468,
  0.0018032014689594892,
  0.01803201468959489,
  0.0018032014689594892,
  0.0018032014689594892,
  0.005409604406878468,
  0.2290065865578551,
  0.04147363378606825,
  0.07573446169629855,
  0.1334369087030022,
  0.34621468204022193,
  0.037867230848149275,
  0.050489641130865695,
  0.14966572192363758,
  0.16769773661323248,
  0.0054096